In [2]:
import requests
import cchardet
url = "https://www.sakeno.com/all_meigara.html"
response = requests.get(url)
response.encoding = cchardet.detect(response.content)["encoding"]
response.status_code

200

In [3]:
from bs4 import BeautifulSoup
bs = BeautifulSoup(response.content,"lxml")

In [4]:
bs = bs.select("#maincontent")[0]

In [5]:
meigara = {}
for p in bs.select("p"):
    ken = p.select("b a")[0].text
    meigara[ken] = []
    for index,a in enumerate(p.select("a")):
        if index != 0:
            meigara[ken].append({"brand":a.text,"href":"https://www.sakeno.com/"+a["href"]})
meigara["北海道"]

[{'brand': '上川大雪', 'href': 'https://www.sakeno.com/meigara/3823'},
 {'brand': '二世古', 'href': 'https://www.sakeno.com/meigara/1234'},
 {'brand': '京極', 'href': 'https://www.sakeno.com/meigara/1235'},
 {'brand': '侍', 'href': 'https://www.sakeno.com/meigara/3661'},
 {'brand': '北の誉', 'href': 'https://www.sakeno.com/meigara/4'},
 {'brand': '大雪乃蔵', 'href': 'https://www.sakeno.com/meigara/8'},
 {'brand': '国稀', 'href': 'https://www.sakeno.com/meigara/9'},
 {'brand': 'まる田', 'href': 'https://www.sakeno.com/meigara/3820'},
 {'brand': '冬花火', 'href': 'https://www.sakeno.com/meigara/1892'},
 {'brand': '北の錦', 'href': 'https://www.sakeno.com/meigara/6'},
 {'brand': '北斗随想', 'href': 'https://www.sakeno.com/meigara/1891'},
 {'brand': '北宝', 'href': 'https://www.sakeno.com/meigara/1448'},
 {'brand': '小樽の女', 'href': 'https://www.sakeno.com/meigara/1710'},
 {'brand': '熊古露里', 'href': 'https://www.sakeno.com/meigara/1449'},
 {'brand': '雪中花（北海道）', 'href': 'https://www.sakeno.com/meigara/1709'},
 {'brand': '北見寒菊'

In [6]:
meigara["北海道"][0]["href"]

'https://www.sakeno.com/meigara/3823'

In [7]:
import re
from time import sleep
import time
from pprint import pprint
from IPython.display import clear_output
import json

In [8]:
def kihonGet(kihon):
    kihonDict = {}
    for k in kihon:
        pair = {}
        if("銘柄名" in k.text or "酒蔵名" in k.text):
            ##かな取得
            tmp = k.select("rt")[0].text
            ##かな消す
            k.select("h3")[0].rt.clear()
            ##漢字取得
            tmp2 = k.select("h3")[0].text
            pair[k.select("th")[0].text]=tmp2 
            pair[k.select("th")[0].text+"_kana"]=tmp
        elif("住所" in k.text):
            ken = ""
            area = ""
            otherUrls = []
            aElems = k.select("a")
            for a in aElems:
                if "https://www.sakeno.com/all_kuramoto_todou" in a["href"]:
                    ken = a.text
                elif "https://www.sakeno.com/search_area.php" in a["href"]:
                    area = a.text
                elif "https://www.google.co.jp/maps/place" in a["href"]:
                    a.parent.clear()
                elif "https://www.sakeno.com/area" in a["href"]:
                    a.clear()
                else:
                    otherUrls.append(a["href"])
            plane = k.select("td")[0].text.replace(u"\xa0",u"")
            pair["ken"]=ken
            pair["area"]=area
            ##TEL
            mat_tmp = re.search('TEL0\d{1,4}-\d{1,4}-\d{4}',plane)
            if mat_tmp != None:
                pair["TEL"] = mat_tmp.group(0).replace("TEL","")
                plane = plane.replace(mat_tmp.group(0),"")
            ##FAX
            mat_tmp = re.search('FAX0\d{1,4}-\d{1,4}-\d{4}',plane)
            if mat_tmp != None:
                pair["FAX"] = mat_tmp.group(0).replace("FAX","")
                plane = plane.replace(mat_tmp.group(0),"")
            ##URL
            mat_tmp = re.search('https?://[\w/:%#\$&\?\(\)~\.=\+\-]+',plane)
            if mat_tmp != None:
                pair["URL"] = mat_tmp.group(0)
                plane = plane.replace(mat_tmp.group(0),"")
            ##その他
            pair[k.select("th")[0].text]=plane
        elif("酒蔵紹介" in k.text):
            pair[k.select("th")[0].text]=k.select("td")[0].text
        elif("銘柄紹介" in k.text):
            pair[k.select("th")[0].text]=k.select("td")[0].text
        elif(k.select("th")!=[] and k.select("td")!=[] ):
            pair[k.select("th")[0].text]=k.select("td")[0].text
        kihonDict.update(pair)
    return kihonDict

In [9]:
def osakeGet(osakeTable):
    for ad in osakeTable.select("div"):
        ad.decompose()
    osakeNameElem = osakeTable.select("th.hyoji_osakename")
    osakeName = [nme.text for nme in osakeNameElem]
    osake = [ nme.parent for nme in osakeNameElem] ##各お酒のｔｒ要素
    
    osakeData = []
    
    for o,on in zip(osake,osakeName):
        tmp = o.select(".osakedata")[0].select("tr")
        od = {}
        ##osakename
        od["name"] = on
        for t in tmp:
            head = t.select("th")[0].text
            data = t.select("td")[0].text
            ##不要文字とる
            head = head.replace("（参考）","")
            if head == "原料米":
                ### % 除外
                data = re.sub(r'(?:[+±　度＋]|（(?:-(?:淡麗～\+濃醇|辛口～\+甘口)|参考)）|\\[nr]|以上|未満|前後)','',data)
            if head == "原料米":
                ### % 除外
                data = re.sub(r'(?:[+±　度＋]|（(?:-(?:淡麗～\+濃醇|辛口～\+甘口)|参考)）|\\[nr]|以上|未満)','',data)
            elif "紹介" in head:
                data = data ##なにもしない            
            else:
                data = re.sub(r'(?:[%+±　度％＋]|（(?:-(?:淡麗～\+濃醇|辛口～\+甘口)|参考)）|\\[nr]|以上|未満|前後)','',data)
            ##～や,で分割する
            if "紹介" in head:
                data = data ##なにもしない 
            else:
                data = re.split(r'[~、，～]',data)
                
            ##1個の物は配列にしない
            if(len(data) < 2):
                data = data[0]
                
            od[head] = data
        osakeData.append(od)

    return osakeData


In [10]:
re.sub(r'(?:[%+±　度％＋]|（(?:-(?:淡麗～\+濃醇|辛口～\+甘口)|参考)）|\\[nr]|以上|未満)','',"-0.06（-辛口～+甘口）")

'-0.06'

In [11]:
if "sakeobj" in locals():
    print("sakeobjあります")
else:
    print("sakeobj無いので作ります")
    sakeobj = {}

sakeobj無いので作ります


In [13]:

todou = input()
sleeptime = 30

if todou not in meigara:
    print("todou not in meigara")
else:

    sakeobj[todou] = []

    # 時間計測開始
    time_sta = time.time()
    time_end = time_sta 

    maxLen = len(meigara[todou])
    for i , meigaraTodou in enumerate(meigara[todou]):
        tim = round((time_end-time_sta)/60,1)
        yosou = round((maxLen-i)*(sleeptime+5)/60,1)
        print( str(i) + "/" + str(maxLen) +"  restTime:" + str(yosou) +"min" +" realTime:"+ str(tim))
        response = requests.get(meigaraTodou["href"])
        response.encoding = cchardet.detect(response.content)["encoding"]
        print(response.status_code)
        bs_m = BeautifulSoup(response.content,"lxml")

        ##各銘柄(brand)の情報がここに収まる
        brandDict = {}
        ##基本情報
        kihon = bs_m.select("table.hyoji")[0]
        brandDict = kihonGet(kihon)
        brandDict['参照']=meigaraTodou["href"]
        ##お酒
        osakeTable = bs_m.select("table.hyoji")[1]
        brandDict["osake"] = osakeGet(osakeTable)

        sakeobj[todou].append(brandDict)

        pprint(brandDict)

        time.sleep(sleeptime)
        # 時間計測終了
        time_end = time.time()

        clear_output()
    print("成功！！  予想経過時間：" + str(round(maxLen*(sleeptime+5)/60,1)) + "min   総経過時間：" + str(tim) + "min")


    #JSON ファイルの読み込み
    f = open('test2.json', 'r')
    json_dict = json.load(f)
    print('json_dict:{}'.format(type(json_dict)))
    f.close

    if todou in json_dict:
        print(todou + "存在します。追加？ y/n")
    else:
        print(todou + "新規追加？ y/n")
    writeFlag = input()
    if writeFlag == "y" or writeFlag == "ｙ":
        ##追加
        json_dict[todou] = sakeobj[todou]

        #JSON データの変換
        print('-----辞書型から JSON 形式の文字列へ変換-----')
        json_str = json.dumps(json_dict)
        print('json_str:{}'.format(type(json_str)))

        print('-----JSON 形式の文字列から辞書型へ変換-----')
        json_dict2 = json.loads(json_str)
        print('json_dict2:{}'.format(type(json_dict2)))

        #JSON データの書き込み
        f2 = open('test2.json', 'w')
        json.dump(json_dict2, f2)
        f2.close()
        print("追加完了")
    else:
        print("追加しませんでした。")

成功！！  予想経過時間：58.3min   総経過時間：51.7min
json_dict:<class 'dict'>
埼玉県新規追加？ y/n


 y


-----辞書型から JSON 形式の文字列へ変換-----
json_str:<class 'str'>
-----JSON 形式の文字列から辞書型へ変換-----
json_dict2:<class 'dict'>
追加完了


In [13]:
f.close()

In [37]:
#JSON ファイルの読み込み
f = open('test2.json', 'r')
json_dict = json.load(f)
print('json_dict:{}'.format(type(json_dict)))
f.close

if todou in json_dict:
    print(todou + "存在します。追加？ y/n")
else:
    print(todou + "新規追加？ y/n")
writeFlag = input()
if writeFlag == "y":
    ##追加
    json_dict[todou] = sakeobj[todou]

    #JSON データの変換
    print('-----辞書型から JSON 形式の文字列へ変換-----')
    json_str = json.dumps(json_dict)
    print('json_str:{}'.format(type(json_str)))

    print('-----JSON 形式の文字列から辞書型へ変換-----')
    json_dict2 = json.loads(json_str)
    print('json_dict2:{}'.format(type(json_dict2)))

    #JSON データの書き込み
    f2 = open('test2.json', 'w')
    json.dump(json_dict2, f2)
    f2.close()
else:
    print("追加しませんでした。")

-----辞書型から JSON 形式の文字列へ変換-----
json_str:<class 'str'>
-----JSON 形式の文字列から辞書型へ変換-----
json_dict2:<class 'dict'>
